In [168]:
import pandas as pd

In [169]:
import tensorflow as tf
from keras import backend as K

In [170]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [171]:
from sklearn.model_selection import train_test_split

In [172]:
df = pd.read_csv('Selected.csv')

In [173]:
df1 = df[df.record < 833508]
df2 = df[df.record >= 833508]

In [174]:
df1 = df1.set_index('record')
df2 = df2.set_index('record')

In [175]:
X = df1.drop('fraud_label', axis = 1)
Y = df1['fraud_label']

In [176]:
x_oot = df2.drop('fraud_label', axis = 1)
y_oot = df2['fraud_label']

In [177]:
df1.head()

,address-homephone-zip5_count180_date,address-homephone-zip5_count7_date,address-homephone-zip5_pastday,address-homephone_count180_date,address-homephone_pastday,address-zip5_0_count180_count_ratio,address-zip5_0_count30_count_ratio,address-zip5_0_count7_count_ratio,address-zip5_count30_date,address-zip5_pastday,dob-name_count30_date,dob-name_pastday,homephone-zip5_count30_date,ssn-dob_pastday,ssn-name_count180_date,ssn-name_count7_date,ssn-name_pastday,ssn_count180_date,ssn_unique_address-zip5,fraud_label
record,,,,,,,,,,,,,,,,,,,,
1,1,1,365.0,1,365.0,0.005556,0.033333,0.142857,1,365.0,1,365.0,1,365.0,1,1,365.0,1,1,0
2,1,1,365.0,1,365.0,0.005556,0.033333,0.142857,1,365.0,1,365.0,1,365.0,1,1,365.0,1,1,1
3,1,1,365.0,1,365.0,0.005556,0.033333,0.142857,1,365.0,1,365.0,1,365.0,1,1,365.0,1,1,0
4,1,1,365.0,1,365.0,0.005556,0.033333,0.142857,1,365.0,1,365.0,1,365.0,1,1,365.0,1,1,0
5,1,1,365.0,1,365.0,0.005556,0.033333,0.142857,1,365.0,1,365.0,1,365.0,1,1,365.0,1,1,0


In [178]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size = 0.2)

In [179]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()

In [180]:
x_train=scaler.fit_transform(x_train.to_numpy())
x_train=pd.DataFrame(x_train,columns=X.columns)

In [181]:
x_test=scaler.fit_transform(x_test.to_numpy())
x_test=pd.DataFrame(x_test,columns=X.columns)

In [182]:
x_oot=scaler.fit_transform(x_oot.to_numpy())
x_oot=pd.DataFrame(x_oot,columns=X.columns)

In [183]:
len(x_oot)

166493

## Try

In [184]:
import numpy as np
model = np.nan

In [185]:
keras.backend.clear_session()

In [186]:
model = Sequential()
model.add(Dense(16, input_dim=19, activation='relu'))
model.add(Dense(12, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [187]:
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [188]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])

In [ ]:
history = model.fit(x_train, y_train, epochs=10,validation_data=(x_test,y_test), batch_size=6400)

Train on 666805 samples, validate on 166702 samples
Epoch 1/10
666805/666805 [==============================] - 13s 20us/step - loss: 0.6581 - acc: 0.6580 - auc: 0.6637 - val_loss: 0.5171 - val_acc: 0.9891 - val_auc: 0.6104
Epoch 2/10
666805/666805 [==============================] - 12s 17us/step - loss: 0.4007 - acc: 0.9898 - auc: 0.6213 - val_loss: 0.2523 - val_acc: 0.9902 - val_auc: 0.6377
Epoch 3/10
666805/666805 [==============================] - 14s 20us/step - loss: 0.1547 - acc: 0.9904 - auc: 0.6532 - val_loss: 0.0913 - val_acc: 0.9904 - val_auc: 0.6688
Epoch 4/10
666805/666805 [==============================] - 14s 21us/step - loss: 0.0701 - acc: 0.9906 - auc: 0.6824 - val_loss: 0.0581 - val_acc: 0.9906 - val_auc: 0.6944
Epoch 5/10
666805/666805 [==============================] - 12s 18us/step - loss: 0.0528 - acc: 0.9910 - auc: 0.7038 - val_loss: 0.0505 - val_acc: 0.9910 - val_auc: 0.7116
Epoch 6/10
666805/666805 [==============================] - 11s 16us/step - loss: 0.0485

In [ ]:
def FDR(x,y,model):
    pred = model.predict(x)
    y_df = pd.DataFrame(y)
    y_df['pred'] = pred
    top = int(len(y_df) * 0.03)
    numbads = sum(y_df['fraud_label'] == 1)
    fdr = y_df.sort_values(by = 'pred', ascending = False).head(top).fraud_label.sum()/numbads
    return fdr

In [ ]:
FDR(x_oot,y_oot,model)

In [ ]:
pred = model.predict(x_train)

In [ ]:
pred

## Loop to find the best parameters

In [123]:
result = pd.DataFrame(columns = ['first_layer','second_layer', 'epoch','batchsize','train_fdr','test_fdr','oot_fdr' ])

In [124]:
def train_model(first_layer, second_layer, epoch, batchsize):
    model = Sequential()
    model.add(Dense(first_layer, input_dim=19, activation='relu'))
    model.add(Dense(second_layer, activation='relu'))
    model.add(Dense(output_dim = 1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', auc])
    model.fit(x_train, y_train, epochs=epoch,validation_data=(x_test,y_test),batch_size=batchsize )
    return model

In [125]:
for first_layer in [16,10]:
    for second_layer in [12,7]:
        for epoch in [10,20,30]:
            for batchsize in [6400,3200]:
                keras.backend.clear_session()
                m = train_model(first_layer, second_layer, epoch, batchsize)
                fdr_train = FDR(x_train,y_train,m)
                fdr_test = FDR(x_test,y_test,m)
                fdr_oot = FDR(x_oot,y_oot,m)
                result = result.append({'first_layer':first_layer,'second_layer':second_layer, 'epoch':epoch,'batchsize':batchsize,'train_fdr':fdr_train,'test_fdr':fdr_test,'oot_fdr':fdr_oot}, ignore_index = True)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="sigmoid", units=1)`
  """


Train on 666805 samples, validate on 166702 samples
Epoch 1/10
666805/666805 [==============================] - 11s 17us/step - loss: 0.4374 - acc: 0.8907 - auc: 0.4359 - val_loss: 0.2094 - val_acc: 0.9902 - val_auc: 0.5953
Epoch 2/10
666805/666805 [==============================] - 9s 14us/step - loss: 0.0968 - acc: 0.9905 - auc: 0.6537 - val_loss: 0.0508 - val_acc: 0.9907 - val_auc: 0.6918
Epoch 3/10
666805/666805 [==============================] - 11s 16us/step - loss: 0.0473 - acc: 0.9908 - auc: 0.7108 - val_loss: 0.0472 - val_acc: 0.9908 - val_auc: 0.7243
Epoch 4/10
666805/666805 [==============================] - 10s 15us/step - loss: 0.0463 - acc: 0.9910 - auc: 0.7316 - val_loss: 0.0469 - val_acc: 0.9909 - val_auc: 0.7378
Epoch 5/10
666805/666805 [==============================] - 10s 15us/step - loss: 0.0460 - acc: 0.9911 - auc: 0.7416 - val_loss: 0.0468 - val_acc: 0.9910 - val_auc: 0.7449
Epoch 6/10
666805/666805 [==============================] - 10s 16us/step - loss: 0.0459 

Epoch 4/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0462 - acc: 0.9911 - auc: 0.7459 - val_loss: 0.0468 - val_acc: 0.9911 - val_auc: 0.7492
Epoch 5/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0460 - acc: 0.9912 - auc: 0.7513 - val_loss: 0.0467 - val_acc: 0.9911 - val_auc: 0.7529
Epoch 6/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0458 - acc: 0.9912 - auc: 0.7538 - val_loss: 0.0467 - val_acc: 0.9910 - val_auc: 0.7554
Epoch 7/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0456 - acc: 0.9913 - auc: 0.7560 - val_loss: 0.0465 - val_acc: 0.9911 - val_auc: 0.7571
Epoch 8/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0455 - acc: 0.9913 - auc: 0.7578 - val_loss: 0.0464 - val_acc: 0.9911 - val_auc: 0.7585
Epoch 9/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0454 - acc: 0.9913 - auc: 0.7589 - val_loss: 0.0463 - va

Train on 666805 samples, validate on 166702 samples
Epoch 1/20
666805/666805 [==============================] - 12s 18us/step - loss: 0.2899 - acc: 0.9824 - auc: 0.5345 - val_loss: 0.1092 - val_acc: 0.9896 - val_auc: 0.6358
Epoch 2/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0716 - acc: 0.9898 - auc: 0.6715 - val_loss: 0.0558 - val_acc: 0.9902 - val_auc: 0.6999
Epoch 3/20
666805/666805 [==============================] - 10s 16us/step - loss: 0.0514 - acc: 0.9903 - auc: 0.7144 - val_loss: 0.0496 - val_acc: 0.9905 - val_auc: 0.7257
Epoch 4/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0481 - acc: 0.9906 - auc: 0.7317 - val_loss: 0.0482 - val_acc: 0.9906 - val_auc: 0.7374
Epoch 5/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0471 - acc: 0.9910 - auc: 0.7411 - val_loss: 0.0477 - val_acc: 0.9909 - val_auc: 0.7437
Epoch 6/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0467

666805/666805 [==============================] - 10s 15us/step - loss: 0.0448 - acc: 0.9914 - auc: 0.7628 - val_loss: 0.0458 - val_acc: 0.9912 - val_auc: 0.7631
Epoch 25/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0448 - acc: 0.9914 - auc: 0.7632 - val_loss: 0.0457 - val_acc: 0.9912 - val_auc: 0.7633
Epoch 26/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0447 - acc: 0.9914 - auc: 0.7634 - val_loss: 0.0457 - val_acc: 0.9912 - val_auc: 0.7636
Epoch 27/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0447 - acc: 0.9914 - auc: 0.7636 - val_loss: 0.0458 - val_acc: 0.9911 - val_auc: 0.7637
Epoch 28/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0447 - acc: 0.9914 - auc: 0.7638 - val_loss: 0.0457 - val_acc: 0.9911 - val_auc: 0.7639
Epoch 29/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0447 - acc: 0.9914 - auc: 0.7640 - val_loss: 0.0458 - val_acc:

Train on 666805 samples, validate on 166702 samples
Epoch 1/20
666805/666805 [==============================] - 11s 17us/step - loss: 0.2034 - acc: 0.9874 - auc: 0.3797 - val_loss: 0.0547 - val_acc: 0.9902 - val_auc: 0.6030
Epoch 2/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0492 - acc: 0.9905 - auc: 0.6676 - val_loss: 0.0481 - val_acc: 0.9909 - val_auc: 0.7022
Epoch 3/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0472 - acc: 0.9910 - auc: 0.7168 - val_loss: 0.0477 - val_acc: 0.9909 - val_auc: 0.7273
Epoch 4/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0468 - acc: 0.9912 - auc: 0.7326 - val_loss: 0.0474 - val_acc: 0.9911 - val_auc: 0.7379
Epoch 5/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0465 - acc: 0.9912 - auc: 0.7410 - val_loss: 0.0472 - val_acc: 0.9911 - val_auc: 0.7436
Epoch 6/20
666805/666805 [==============================] - 10s 15us/step - loss: 0.0463

Epoch 28/30
666805/666805 [==============================] - 10s 14us/step - loss: 0.0451 - acc: 0.9914 - auc: 0.7540 - val_loss: 0.0462 - val_acc: 0.9911 - val_auc: 0.7543
Epoch 29/30
666805/666805 [==============================] - 9s 14us/step - loss: 0.0451 - acc: 0.9914 - auc: 0.7545 - val_loss: 0.0461 - val_acc: 0.9911 - val_auc: 0.7547
Epoch 30/30
666805/666805 [==============================] - 9s 14us/step - loss: 0.0451 - acc: 0.9914 - auc: 0.7549 - val_loss: 0.0461 - val_acc: 0.9911 - val_auc: 0.7551
Train on 666805 samples, validate on 166702 samples
Epoch 1/30
666805/666805 [==============================] - 11s 17us/step - loss: 0.2904 - acc: 0.9315 - auc: 0.4325 - val_loss: 0.0614 - val_acc: 0.9903 - val_auc: 0.6303
Epoch 2/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0515 - acc: 0.9905 - auc: 0.6835 - val_loss: 0.0494 - val_acc: 0.9907 - val_auc: 0.7098
Epoch 3/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.047

Train on 666805 samples, validate on 166702 samples
Epoch 1/30
666805/666805 [==============================] - 11s 16us/step - loss: 0.3451 - acc: 0.9593 - auc: 0.6175 - val_loss: 0.1352 - val_acc: 0.9872 - val_auc: 0.6565
Epoch 2/30
666805/666805 [==============================] - 9s 14us/step - loss: 0.0860 - acc: 0.9870 - auc: 0.6705 - val_loss: 0.0642 - val_acc: 0.9878 - val_auc: 0.6881
Epoch 3/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0580 - acc: 0.9881 - auc: 0.6995 - val_loss: 0.0550 - val_acc: 0.9882 - val_auc: 0.7093
Epoch 4/30
666805/666805 [==============================] - 10s 15us/step - loss: 0.0532 - acc: 0.9881 - auc: 0.7164 - val_loss: 0.0525 - val_acc: 0.9882 - val_auc: 0.7220
Epoch 5/30
666805/666805 [==============================] - 9s 14us/step - loss: 0.0515 - acc: 0.9883 - auc: 0.7262 - val_loss: 0.0513 - val_acc: 0.9885 - val_auc: 0.7303
Epoch 6/30
666805/666805 [==============================] - 9s 14us/step - loss: 0.0505 - 

Epoch 18/30
666805/666805 [==============================] - 11s 16us/step - loss: 0.0453 - acc: 0.9914 - auc: 0.7595 - val_loss: 0.0462 - val_acc: 0.9912 - val_auc: 0.7598
Epoch 19/30
666805/666805 [==============================] - 11s 17us/step - loss: 0.0452 - acc: 0.9914 - auc: 0.7599 - val_loss: 0.0461 - val_acc: 0.9912 - val_auc: 0.7601
Epoch 20/30
666805/666805 [==============================] - 11s 16us/step - loss: 0.0452 - acc: 0.9914 - auc: 0.7603 - val_loss: 0.0461 - val_acc: 0.9912 - val_auc: 0.7605
Epoch 21/30
666805/666805 [==============================] - 11s 17us/step - loss: 0.0451 - acc: 0.9914 - auc: 0.7606 - val_loss: 0.0461 - val_acc: 0.9912 - val_auc: 0.7608
Epoch 22/30
666805/666805 [==============================] - 11s 16us/step - loss: 0.0451 - acc: 0.9914 - auc: 0.7610 - val_loss: 0.0460 - val_acc: 0.9912 - val_auc: 0.7611
Epoch 23/30
666805/666805 [==============================] - 11s 16us/step - loss: 0.0451 - acc: 0.9914 - auc: 0.7612 - val_loss: 0.046

In [126]:
result

,first_layer,second_layer,epoch,batchsize,train_fdr,test_fdr,oot_fdr
0,16.0,12.0,10.0,6400.0,0.537544,0.516279,0.515088
1,16.0,12.0,10.0,3200.0,0.534744,0.516279,0.515926
2,16.0,12.0,20.0,6400.0,0.538996,0.517548,0.517603
3,16.0,12.0,20.0,3200.0,0.538063,0.519239,0.519698
4,16.0,12.0,30.0,6400.0,0.538270,0.517970,0.515507
5,16.0,12.0,30.0,3200.0,0.538477,0.518393,0.522213
6,16.0,7.0,10.0,6400.0,0.529247,0.509091,0.500000
7,16.0,7.0,10.0,3200.0,0.529973,0.509937,0.503772
8,16.0,7.0,20.0,6400.0,0.533603,0.512896,0.511316
9,16.0,7.0,20.0,3200.0,0.536611,0.515011,0.515507


In [136]:
result.to_csv('result_neural_net.csv') 